In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete3O0X7L/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete3O0X7L/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete3O0X7L/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
train_data, test_data = imdb['train'], imdb['test']

In [4]:
train_sentences = []
train_labels = []
test_sentences = []
test_labels = []

for s,l in train_data:
    train_sentences.append(s.numpy().decode('utf8'))
    train_labels.append(l.numpy())
    
for s,l in test_data:
    test_sentences.append(s.numpy().decode('utf8'))
    test_labels.append(l.numpy())

In [5]:
len(train_sentences), len(train_labels), len(test_sentences), len(test_labels)

(25000, 25000, 25000, 25000)

In [6]:
train_labels_final = np.array(train_labels)
test_labels_final = np.array(test_labels)

In [7]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trun_type = 'post'
oov_token = '<oov>'
num_epochs = 10

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trun_type)

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, maxlen=max_length)

In [9]:
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Flatten(),
    keras.layers.Dense(6, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 6)                 11526     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171533 (670.05 KB)
Trainable params: 171533 (670.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
model.fit(padded, train_labels_final,
         epochs=num_epochs, validation_data=(test_padded, test_labels_final))

Epoch 1/10
782/782 [==============================] - 6s 7ms/step - loss: 0.4902 - accuracy: 0.7429 - val_loss: 0.3471 - val_accuracy: 0.8472
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2444 - accuracy: 0.9055 - val_loss: 0.3565 - val_accuracy: 0.8464
Epoch 3/10
782/782 [==============================] - 4s 6ms/step - loss: 0.1081 - accuracy: 0.9685 - val_loss: 0.4411 - val_accuracy: 0.8295
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0282 - accuracy: 0.9961 - val_loss: 0.5300 - val_accuracy: 0.8267
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0075 - accuracy: 0.9994 - val_loss: 0.6009 - val_accuracy: 0.8256
Epoch 6/10
782/782 [==============================] - 4s 6ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.6352 - val_accuracy: 0.8313
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 9.6310e-04 - accuracy: 1.0000 - val_loss: 0.6794 - val_accuracy: 0.8318
Ep

In [11]:
embeddings_layer = model.layers[0]
embedding_weights = embeddings_layer.weights[0]

In [12]:
# Get the index-word dictionary
reverse_word_index = tokenizer.index_word

In [13]:
import io

out_v = io.open('vecs.tsv','w',encoding='utf8')
out_m = io.open('meta.tsv','w',encoding='utf8')

for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    out_m.write(word+"\n")
    
    embedding = embedding_weights[word_num]
    out_v.write("\t".join(str(x) for x in embedding)+"\n")

out_v.close()
out_m.close()